In [97]:
# Importing necessary packages + initializing settings
import pandas as pd
import nltk
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from collections import Counter
from string import punctuation
nltk.download(["stopwords","vader_lexicon"])
import matplotlib.pyplot as plt

pd.set_option("display.max_rows",5)

# Loading data
# training_data = pd.read_csv("./twitter_training.csv", index_col=0, names=["Index","Entity","Sentiment","Tweet"])
testing_data = pd.read_csv("./twitter_validation.csv", index_col=0, names=["Index","Entity","Sentiment","Tweet"])

# Outputting first 5 rows of training data
# training_data.head()
# Outputting first 5 rows of testing data
# testing_data.head()

    #removing_mentions = r'@[A-Za-z0-9_]+'
    #removing_links = r'httpss?://[^ ]+'
    #removing_links_mentions = r'|'.join((removing_mentions,removing_links))
    #removing_www = r'www.[^ ]+'

indices = testing_data.index.values.tolist()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\layal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\layal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [115]:
def clean_tweet(tweet):

    removing_mentions = r'@[A-Za-z0-9_]+'

    tweet = re.sub(removing_mentions, '', tweet)

    tokens = tweet.split()
    table = str.maketrans(" "," ",punctuation)
    tokens = [w.translate(table) for w in tokens]
    token = [w for w in tokens if w.isalpha()]
    stop_words = stopwords.words("english")
    new_tweet = [w for w in token if w.lower() not in stop_words]

    return ' '.join(new_tweet)

In [112]:
testing_data.Tweet

Index
3364    I mentioned on Facebook that I was struggling ...
352     BBC News - Amazon boss Jeff Bezos rejects clai...
                              ...                        
8069    Bought a fraction of Microsoft today. Small wins.
6960    Johnson & Johnson to stop selling talc baby po...
Name: Tweet, Length: 1000, dtype: object

In [113]:
# Testing tweet cleaner
idx = indices[900]

# Original input
testing_data.Tweet.loc[idx]

'The iPhone 12 needs to hurry up cause I need to get it with @Verizon cause @sprint is trash ever since the merger with @TMobile! Like my speed is from the Middle Ages!! #iPhone12'

In [116]:
# Clean output
clean_tweet(testing_data.Tweet.loc[idx])

'iPhone needs hurry cause need get cause trash ever since merger Like speed Middle Ages'

In [117]:
token_tweets = []

for idx in testing_data.index.values.tolist():
    token_tweets.append(clean_tweet(testing_data.Tweet.loc[idx]))
    


In [118]:
freq_dist_tweet = nltk.FreqDist(token_tweets)
freq_dist_tweet.most_common(3)

[('Wow', 3),
 ('Check video LeagueofLegends Captured Outplayed', 3),
 ('Red Dead Redemption', 2)]

In [119]:
# Testing sentiment analyzer
idx = 6960
sentiment_analyzer = SentimentIntensityAnalyzer()
print(sentiment_analyzer.polarity_scores(clean_tweet(testing_data.Tweet.loc[idx])))
print(testing_data.Sentiment.loc[idx])


{'neg': 0.196, 'neu': 0.804, 'pos': 0.0, 'compound': -0.296}
Neutral


In [122]:
all_sentiment = []
count = 0

pos_count_vader = 0
neg_count_vader = 0
neu_count_vader = 0

pos_count_true = 0
neg_count_true = 0
neu_count_true = 0


for idx in testing_data.index.values.tolist():
    input_tweet = clean_tweet(testing_data.Tweet.loc[idx])
    all_sentiment.append(sentiment_analyzer.polarity_scores(input_tweet))
    
    score = all_sentiment[count]

    # Not sure how to threshold positive vs negative vs neutral score
    if score['compound'] > 0.5:
        pos_count_vader+=1
    elif score['compound'] < -0.5:
        neg_count_vader+=1
    else:
        neu_count_vader+=1
    
    if testing_data.Sentiment.loc[idx]=='Positive':
        pos_count_true+=1
    elif testing_data.Sentiment.loc[idx]=='Negative':
        neg_count_true+=1
    else:
        neu_count_true+=1

    count+=1

print(pos_count_vader,neg_count_vader,neu_count_vader)
print(pos_count_true,neg_count_true,neu_count_true)


322 228 450
277 266 457
